# 📚 Workshop: RAG + LangChain + Streamlit

## ช่วงที่ 2: Data Ingestion & Vector Store (0:30 – 1:30)

---

### 🎯 วัตถุประสงค์การเรียนรู้
- เรียนรู้การเตรียมข้อมูล (PDF, Text, Website)
- ใช้ LangChain DocumentLoader
- แบ่งเอกสารด้วย RecursiveCharacterTextSplitter
- สร้าง embeddings (HuggingFaceEmbeddings)
- เก็บลง vector database (FAISS, Qdrant)

---


## 📄 การเตรียมข้อมูล (Data Preparation)

### ประเภทข้อมูลที่รองรับ:

1. **PDF Documents** 📄
   - เอกสาร PDF ทั่วไป
   - PDF ที่มีตาราง
   - PDF ที่สแกน (OCR)

2. **Text Files** 📝
   - .txt files
   - .md files
   - .csv files

3. **Web Content** 🌐
   - Web pages
   - Wikipedia articles
   - Blog posts

4. **Structured Data** 📊
   - JSON files
   - XML files
   - Database exports

### ตัวอย่างข้อมูลที่เราจะใช้:
- `pdf/จังหวัดน่าน.pdf` - ข้อมูลจังหวัดน่าน
- `pdf/CV.pdf` - ข้อมูลส่วนตัว


In [8]:
# ติดตั้งและ Import Libraries ที่จำเป็น
import os
import sys
from pathlib import Path

# เพิ่ม path สำหรับ import modules
sys.path.append('..')

# Import LangChain components
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS, Qdrant
from langchain.schema import Document

# Import utilities
from dotenv import load_dotenv
import logging

# ตั้งค่า logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# โหลด environment variables
load_dotenv()

print("✅ Libraries imported successfully!")
print(f"📁 Current directory: {os.getcwd()}")
print(f"📄 PDF directory exists: {os.path.exists('../pdf')}")


✅ Libraries imported successfully!
📁 Current directory: /home/game/01_Projects/workshop_rag_rmutl/notebooks
📄 PDF directory exists: True


## 📥 Document Loader

### LangChain Document Loaders

LangChain มี Document Loaders หลายประเภท:

1. **PyPDFLoader** - สำหรับ PDF files
2. **TextLoader** - สำหรับ text files
3. **CSVLoader** - สำหรับ CSV files
4. **WebBaseLoader** - สำหรับ web content
5. **DirectoryLoader** - สำหรับโฟลเดอร์ทั้งหมด

### ตัวอย่างการใช้งาน:


In [9]:
# โหลด PDF Document
def load_pdf_document(file_path):
    """โหลดเอกสาร PDF และแสดงข้อมูล"""
    try:
        # สร้าง PDF Loader
        loader = PyPDFLoader(file_path)
        
        # โหลดเอกสาร
        documents = loader.load()
        
        print(f"📄 โหลดเอกสาร: {file_path}")
        print(f"📊 จำนวนหน้า: {len(documents)}")
        print(f"📝 จำนวนตัวอักษรทั้งหมด: {sum(len(doc.page_content) for doc in documents)}")
        
        # แสดงตัวอย่างเนื้อหาจากหน้าแรก
        if documents:
            print(f"\n📖 ตัวอย่างเนื้อหาจากหน้าแรก:")
            print("-" * 50)
            print(documents[0].page_content[:500] + "...")
            print("-" * 50)
        
        return documents
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {e}")
        return []

# ทดสอบโหลด PDF
pdf_path = "../pdf/จังหวัดน่าน.pdf"
if os.path.exists(pdf_path):
    documents = load_pdf_document(pdf_path)
else:
    print(f"❌ ไม่พบไฟล์: {pdf_path}")
    documents = []


📄 โหลดเอกสาร: ../pdf/จังหวัดน่าน.pdf
📊 จำนวนหน้า: 6
📝 จำนวนตัวอักษรทั้งหมด: 9363

📖 ตัวอย่างเนื้อหาจากหน้าแรก:
--------------------------------------------------
จังหวัดน่าน 
น่าน เป็นจังหวัดหนึ่งในประเทศไทย ตั้งอยู่ทางทิศตะวันออกสุดของภาคเหนือ เป็นที่ตั้ง
ของเมืองที่สำคัญในอดีต เช่น เวียงวรนคร (เมืองพลัว) เวียงศีรษะเกษ (เมืองงั่ว) เวียงภูเพียงแช่
แห้ง อีกทั้งยังเป็นแหล่งต้นน้ำของแม่น้ำน่าน 
ประวัติศาสตร์  มีประวัติความเป็นมาที่เก่าแก่ยาวนาน มีชื่อเรียกในพงศาวดารว่า นันท
บุรี เมืองน่านในอดีตเป็นนครรัฐเล็ก  ๆ ก่อตัวขึ้นราวกลางพุทธศตวรรษที่  18 บริเวณที่ราบลุ่ม
แม่น้ำน่านและแม่น้ำสาขาในหุบเขาทางตะวันออกของภาคเหนือ 
สมัยเมืองล่าง-วรนคร 
ประวัติศาสตร์เมืองน่...
--------------------------------------------------


## ✂️ Text Splitter

### ทำไมต้องแบ่งเอกสาร?

1. **Context Window Limit**: LLM มีข้อจำกัดในการประมวลผลข้อความยาว
2. **Semantic Search**: การค้นหาที่แม่นยำขึ้นเมื่อแบ่งเป็น chunks เล็กๆ
3. **Memory Efficiency**: ประหยัดหน่วยความจำในการประมวลผล

### RecursiveCharacterTextSplitter

**RecursiveCharacterTextSplitter** แบ่งเอกสารโดยใช้ลำดับความสำคัญของตัวคั่น:

1. `\n\n` (ย่อหน้า)
2. `\n` (บรรทัดใหม่)
3. ` ` (ช่องว่าง)
4. `""` (ตัวอักษร)

### พารามิเตอร์สำคัญ:
- `chunk_size`: ขนาดของแต่ละ chunk (ตัวอักษร)
- `chunk_overlap`: การทับซ้อนระหว่าง chunks


In [10]:
# แบ่งเอกสารเป็น chunks
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """แบ่งเอกสารเป็น chunks"""
    try:
        # สร้าง Text Splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", " ", ""]
        )
        
        # แบ่งเอกสาร
        chunks = text_splitter.split_documents(documents)
        
        print(f"✂️ แบ่งเอกสารเป็น chunks")
        print(f"📊 จำนวน chunks: {len(chunks)}")
        print(f"📏 ขนาด chunk: {chunk_size} ตัวอักษร")
        print(f"🔄 การทับซ้อน: {chunk_overlap} ตัวอักษร")
        
        # แสดงตัวอย่าง chunks
        print(f"\n📖 ตัวอย่าง chunks:")
        print("-" * 50)
        for i, chunk in enumerate(chunks[:3]):  # แสดง 3 chunks แรก
            print(f"Chunk {i+1}:")
            print(f"ความยาว: {len(chunk.page_content)} ตัวอักษร")
            print(f"เนื้อหา: {chunk.page_content[:200]}...")
            print("-" * 30)
        
        return chunks
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {e}")
        return []

# แบ่งเอกสาร
if documents:
    chunks = split_documents(documents, chunk_size=1000, chunk_overlap=200)
else:
    print("❌ ไม่มีเอกสารให้แบ่ง")
    chunks = []


✂️ แบ่งเอกสารเป็น chunks
📊 จำนวน chunks: 13
📏 ขนาด chunk: 1000 ตัวอักษร
🔄 การทับซ้อน: 200 ตัวอักษร

📖 ตัวอย่าง chunks:
--------------------------------------------------
Chunk 1:
ความยาว: 989 ตัวอักษร
เนื้อหา: จังหวัดน่าน 
น่าน เป็นจังหวัดหนึ่งในประเทศไทย ตั้งอยู่ทางทิศตะวันออกสุดของภาคเหนือ เป็นที่ตั้ง
ของเมืองที่สำคัญในอดีต เช่น เวียงวรนคร (เมืองพลัว) เวียงศีรษะเกษ (เมืองงั่ว) เวียงภูเพียงแช่
แห้ง อีกทั้ง...
------------------------------
Chunk 2:
ความยาว: 944 ตัวอักษร
เนื้อหา: เสี้ยว บ้านทุ่งฆ้อง  บ้านลอมกลาง  ตำบลยม อำเภอท่าวังผา ) เพราะปรากฏร่องรอยชุมชนใน
สภาพที่เป็นคูน้ำ  คันดิน และกำแพงเมืองซ้อนกันอยู่  เห็นชัดเจนที่สุดคือบริเวณข้างพระธาตุ
จอมพริกบ้านเสี้ยว มีกำแพงเมือง...
------------------------------
Chunk 3:
ความยาว: 261 ตัวอักษร
เนื้อหา: ครองเมืองย่างแทน จึงให้เสนาอำมาตย์ไปเชิญ เจ้าเก้าเถื่อนเกรงใจปู่จึงยอมไปอยู่เมืองย่างและ
มอบให้ชายาคือนางพญาแม่ท้าวคำปินดูแลรักษาเมืองปัวแทน เมื่อพญาภูคาถึงแก่พิราลัย เจ้า
เก้าเถื่อนจึงครองเมืองย่างแท...
------------------------------


## 🔢 Embeddings

### Embeddings คืออะไร?

**Embeddings** คือการแปลงข้อความเป็นตัวเลข (vectors) ที่สามารถวัดความคล้ายคลึงกันได้

### ประเภทของ Embeddings:

1. **OpenAI Embeddings** - ใช้ OpenAI API (ต้องเสียค่าใช้จ่าย)
2. **HuggingFace Embeddings** - ใช้โมเดลจาก HuggingFace (ฟรี)
3. **Sentence Transformers** - โมเดลเฉพาะสำหรับ embeddings

### HuggingFace Embeddings ที่แนะนำ:

- `all-MiniLM-L6-v2` - ขนาดเล็ก, เร็ว (384 dimensions)
- `all-mpnet-base-v2` - คุณภาพดี (768 dimensions)
- `paraphrase-multilingual-MiniLM-L12-v2` - รองรับหลายภาษา


In [11]:
# สร้าง Embeddings
def create_embeddings(model_name="all-MiniLM-L6-v2"):
    """สร้าง embeddings model"""
    try:
        print(f"🔢 สร้าง Embeddings Model: {model_name}")
        
        # สร้าง HuggingFace Embeddings
        embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            cache_folder="../model_cache"  # เก็บ cache ในโฟลเดอร์ model_cache
        )
        
        print(f"✅ สร้าง Embeddings สำเร็จ")
        print(f"📏 Vector dimensions: {embeddings.client.get_sentence_embedding_dimension()}")
        
        # ทดสอบ embeddings
        test_text = "จังหวัดน่านมีประชากรเท่าไหร่?"
        test_embedding = embeddings.embed_query(test_text)
        print(f"🧪 ทดสอบ embedding: {len(test_embedding)} dimensions")
        
        return embeddings
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {e}")
        return None

# สร้าง embeddings
embeddings = create_embeddings("all-MiniLM-L6-v2")


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


🔢 สร้าง Embeddings Model: all-MiniLM-L6-v2
✅ สร้าง Embeddings สำเร็จ
📏 Vector dimensions: 384
🧪 ทดสอบ embedding: 384 dimensions


## 🗄️ Vector Store

### Vector Store คืออะไร?

**Vector Store** คือฐานข้อมูลที่เก็บและค้นหา vectors (embeddings) ได้อย่างมีประสิทธิภาพ

### ประเภทของ Vector Store:

1. **FAISS** (Facebook AI Similarity Search)
   - ✅ เร็ว, ใช้หน่วยความจำน้อย
   - ✅ รองรับการค้นหาแบบ similarity
   - ❌ เก็บในหน่วยความจำ (ไม่ persistent)

2. **Qdrant**
   - ✅ Persistent storage
   - ✅ รองรับ metadata filtering
   - ✅ API ที่ดี
   - ❌ ต้องติดตั้ง server แยก

3. **Chroma**
   - ✅ ง่ายต่อการใช้งาน
   - ✅ Persistent storage
   - ✅ รองรับ metadata


In [12]:
# สร้าง Vector Store (FAISS)
def create_faiss_vectorstore(chunks, embeddings):
    """สร้าง FAISS Vector Store"""
    try:
        print("🗄️ สร้าง FAISS Vector Store...")
        
        # สร้าง FAISS vector store
        vectorstore = FAISS.from_documents(
            documents=chunks,
            embedding=embeddings
        )
        
        print(f"✅ สร้าง FAISS Vector Store สำเร็จ")
        print(f"📊 จำนวน documents: {vectorstore.index.ntotal}")
        
        # ทดสอบการค้นหา
        test_query = "จังหวัดน่านมีประชากรเท่าไหร่?"
        print(f"\n🔍 ทดสอบการค้นหา: {test_query}")
        
        # ค้นหา documents ที่เกี่ยวข้อง
        similar_docs = vectorstore.similarity_search(test_query, k=3)
        
        print(f"📋 พบ {len(similar_docs)} documents ที่เกี่ยวข้อง:")
        for i, doc in enumerate(similar_docs, 1):
            print(f"\nDocument {i}:")
            print(f"ความยาว: {len(doc.page_content)} ตัวอักษร")
            print(f"เนื้อหา: {doc.page_content[:200]}...")
            print(f"Metadata: {doc.metadata}")
        
        return vectorstore
        
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาด: {e}")
        return None

# สร้าง Vector Store
if chunks and embeddings:
    vectorstore = create_faiss_vectorstore(chunks, embeddings)
else:
    print("❌ ไม่สามารถสร้าง Vector Store ได้ - ตรวจสอบ chunks และ embeddings")
    vectorstore = None


🗄️ สร้าง FAISS Vector Store...


✅ สร้าง FAISS Vector Store สำเร็จ
📊 จำนวน documents: 13

🔍 ทดสอบการค้นหา: จังหวัดน่านมีประชากรเท่าไหร่?
📋 พบ 3 documents ที่เกี่ยวข้อง:

Document 1:
ความยาว: 246 ตัวอักษร
เนื้อหา: สมัยกรุงรัตนโกสินทร์ นครน่านมีฐานะเป็นหัวเมืองประเทศราช เจ้าผู้ครองนครน่านในชั้นหลัง
ทุกองค์ต่างปฏิบัติหน้าที่ราชการด้วยความเที่ยงธรรม มีความซื่อสัตย์ จงรักภักดีต่อ
พระมหากษัตริย์แห่งราชวงศ์จักรี ได้ช...
Metadata: {'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-02-20T03:12:14+07:00', 'author': 'Jeerasak Ananta', 'moddate': '2025-02-20T03:12:14+07:00', 'source': '../pdf/จังหวัดน่าน.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3'}

Document 2:
ความยาว: 261 ตัวอักษร
เนื้อหา: ครองเมืองย่างแทน จึงให้เสนาอำมาตย์ไปเชิญ เจ้าเก้าเถื่อนเกรงใจปู่จึงยอมไปอยู่เมืองย่างและ
มอบให้ชายาคือนางพญาแม่ท้าวคำปินดูแลรักษาเมืองปัวแทน เมื่อพญาภูคาถึงแก่พิราลัย เจ้า
เก้าเถื่อนจึงครองเมืองย่างแท...
Metadata: {'producer': 'Microsoft® Word for Microsoft 365', 'c

## 🔄 เปรียบเทียบ Vector Stores

### ทดสอบ FAISS vs Qdrant


In [13]:
# สร้าง Qdrant Vector Store (ถ้ามี Qdrant server)
def create_qdrant_vectorstore(chunks, embeddings):
    """สร้าง Qdrant Vector Store"""
    try:
        print("🗄️ สร้าง Qdrant Vector Store...")
        
        # สร้าง Qdrant vector store
        vectorstore = Qdrant.from_documents(
            documents=chunks,
            embedding=embeddings,
            collection_name="documents",
            url=":memory:",  # ใช้ in-memory สำหรับ demo
            prefer_grpc=True
        )
        
        print(f"✅ สร้าง Qdrant Vector Store สำเร็จ")
        
        # ทดสอบการค้นหา
        test_query = "จังหวัดน่านมีสถานที่ท่องเที่ยวอะไรบ้าง?"
        print(f"\n🔍 ทดสอบการค้นหา: {test_query}")
        
        similar_docs = vectorstore.similarity_search(test_query, k=2)
        
        print(f"📋 พบ {len(similar_docs)} documents ที่เกี่ยวข้อง:")
        for i, doc in enumerate(similar_docs, 1):
            print(f"\nDocument {i}:")
            print(f"เนื้อหา: {doc.page_content[:150]}...")
        
        return vectorstore
        
    except Exception as e:
        print(f"❌ ไม่สามารถสร้าง Qdrant Vector Store: {e}")
        print("💡 ใช้ FAISS แทน")
        return None

# สร้าง Qdrant Vector Store (ถ้าเป็นไปได้)
if chunks and embeddings:
    qdrant_vectorstore = create_qdrant_vectorstore(chunks, embeddings)
else:
    qdrant_vectorstore = None


🗄️ สร้าง Qdrant Vector Store...
❌ ไม่สามารถสร้าง Qdrant Vector Store: Failed to parse: :memory:
💡 ใช้ FAISS แทน


## 💾 บันทึก Vector Store

### การบันทึก FAISS Vector Store


In [14]:
# บันทึก Vector Store
def save_vectorstore(vectorstore, save_path="../vectorstore"):
    """บันทึก Vector Store"""
    try:
        if vectorstore:
            print(f"💾 บันทึก Vector Store ไปที่: {save_path}")
            vectorstore.save_local(save_path)
            print("✅ บันทึกสำเร็จ")
            return True
        else:
            print("❌ ไม่มี Vector Store ให้บันทึก")
            return False
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดในการบันทึก: {e}")
        return False

# บันทึก Vector Store
if vectorstore:
    save_success = save_vectorstore(vectorstore)
else:
    print("❌ ไม่มี Vector Store ให้บันทึก")


💾 บันทึก Vector Store ไปที่: ../vectorstore
✅ บันทึกสำเร็จ


## 📝 สรุปช่วงที่ 2

### สิ่งที่เราได้เรียนรู้:

1. **Document Loading** 📄
   - ใช้ PyPDFLoader โหลด PDF
   - แสดงข้อมูลเอกสาร (จำนวนหน้า, ตัวอักษร)

2. **Text Splitting** ✂️
   - ใช้ RecursiveCharacterTextSplitter
   - แบ่งเอกสารเป็น chunks พร้อม overlap

3. **Embeddings** 🔢
   - ใช้ HuggingFaceEmbeddings
   - แปลงข้อความเป็น vectors

4. **Vector Store** 🗄️
   - สร้าง FAISS Vector Store
   - ทดสอบการค้นหา similarity search
   - บันทึก Vector Store

### Pipeline ที่สร้างขึ้น:
```
PDF → Document Loader → Text Splitter → Embeddings → Vector Store
```

### ต่อไปในช่วงที่ 3:
เราจะสร้าง RAG Pipeline และเชื่อมต่อกับ LLM

---

**⏰ เวลาที่ใช้: 60 นาที**
**📚 ไฟล์ต่อไป: `03_build_rag_pipeline.ipynb`**
